In [1]:
# Include Pytorch and neural network classes support
import torch 
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

# Include Transforms, Pre-trained networks, Datasets and Datasets loaders
import torchvision
import torchvision.transforms as transforms

# Library that gives support for tensorboard and pytorch
from tensorboardX import SummaryWriter

# Set enviroment variable for make only the first GPU visible
import os
os.environ["CUDA_VISIBLE_DEVICES"] = str(0)
import time
import copy

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Hyper parameters
num_epochs = 25
num_classes = 10
batch_size = 100
learning_rate = 0.001

### Load MNSIT Data
Observe that we can use torchvision to have MNIST

In [2]:
# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='./data_mnist/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./data_mnist/',
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

dataloaders = {'train':train_loader, 'val':test_loader}
dataset_sizes = {'train': len(train_loader), 'val': len(test_loader)}

### Define  a CNN Model

In [3]:
class TwoLayerCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(TwoLayerCNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        #out = out.reshape(out.size(0), -1)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

### Instantiation Model, loss and optimizer

In [4]:
model = TwoLayerCNN(num_classes)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Tensorboard writer at logs directory
writer = SummaryWriter('logs')
# Send graph to tensorboard
writer.add_graph(model, torch.rand(10,1,28,28))
model = model.to(device)

## Generic training Function

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    # For each epoch (1 complete training set run is one epoch)
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:                
                # Send inputs/labels to GPU
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients (Needed before loss.backward and optimizer step)
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
                # Send loss to tensorboard
                writer.add_scalar('loss/'+phase, running_loss, epoch)

            # Get epoch loss and accuracy for the current phase
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model (Copy the best model)
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    # Calculate whole training time
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    # Return best model
    return model

In [ ]:
best_model = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=num_epochs)

Epoch 0/24
----------
train Loss: 15.7552 Acc: 95.7617
val Loss: 6.8491 Acc: 97.8200

Epoch 1/24
----------
train Loss: 4.7601 Acc: 98.5333
